In [1]:
# Importing necessary libraries
import pandas as pd
#pd.set_option('display.max_rows', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split

## Reading the Dataset

In [3]:
# Reading dataset
survival = pd.read_csv('survival_prediction_data.csv')
survival.head()

encounter_id  patient_id  hospital_id   age    bmi  elective_surgery  \
0         66154       25312          118  68.0  22.73                 0   
1        114252       59342           81  77.0  27.42                 0   
2        119783       50777          118  25.0  31.95                 0   
3         79267       46918          118  81.0  22.64                 1   
4         92056       34377           33  19.0    NaN                 0   

   ethnicity gender  height           icu_admit_source  ...  \
0  Caucasian      M   180.3                      Floor  ...   
1  Caucasian      F   160.0                      Floor  ...   
2  Caucasian      F   172.7       Accident & Emergency  ...   
3  Caucasian      F   165.1  Operating Room / Recovery  ...   
4  Caucasian      M   188.0       Accident & Emergency  ...   

   diabetes_mellitus hepatic_failure immunosuppression  leukemia  lymphoma  \
0                1.0             0.0               0.0       0.0       0.0   
1                1.0             0.0               0.0       0.0       0.0   
2                0.0             0.0               0.0       0.0       0.0   
3                0.0             0.0               0.0       0.0       0.0   
4                0.0             0.0               0.0       0.0       0.0   

   solid_tumor_with_metastasis  apache_3j_bodysystem  apache_2_bodysystem  \
0                          0.0                Sepsis       Cardiovascular   
1                          0.0           Respiratory          Respiratory   
2                          0.0             Metabolic            Metabolic   
3                          0.0        Cardiovascular       Cardiovascular   
4                          0.0                Trauma               Trauma   

   Unnamed: 83  hospital_death  
0          NaN               0  
1          NaN               0  
2          NaN               0  
3          NaN               0  
4          NaN               0  

[5 rows x 85 columns]

In [4]:
# Removing unwanted columns
survival.drop(columns=['Unnamed: 83', 'encounter_id', 'patient_id', 'hospital_id', 'icu_id'], inplace=True)
survival.head()

age    bmi  elective_surgery  ethnicity gender  height  \
0  68.0  22.73                 0  Caucasian      M   180.3   
1  77.0  27.42                 0  Caucasian      F   160.0   
2  25.0  31.95                 0  Caucasian      F   172.7   
3  81.0  22.64                 1  Caucasian      F   165.1   
4  19.0    NaN                 0  Caucasian      M   188.0   

            icu_admit_source icu_stay_type      icu_type  pre_icu_los_days  \
0                      Floor         admit         CTICU          0.541667   
1                      Floor         admit  Med-Surg ICU          0.927778   
2       Accident & Emergency         admit  Med-Surg ICU          0.000694   
3  Operating Room / Recovery         admit         CTICU          0.000694   
4       Accident & Emergency         admit  Med-Surg ICU          0.073611   

   ...  cirrhosis  diabetes_mellitus  hepatic_failure  immunosuppression  \
0  ...        0.0                1.0              0.0                0.0   
1  ...        0.0                1.0              0.0                0.0   
2  ...        0.0                0.0              0.0                0.0   
3  ...        0.0                0.0              0.0                0.0   
4  ...        0.0                0.0              0.0                0.0   

   leukemia  lymphoma  solid_tumor_with_metastasis  apache_3j_bodysystem  \
0       0.0       0.0                          0.0                Sepsis   
1       0.0       0.0                          0.0           Respiratory   
2       0.0       0.0                          0.0             Metabolic   
3       0.0       0.0                          0.0        Cardiovascular   
4       0.0       0.0                          0.0                Trauma   

   apache_2_bodysystem  hospital_death  
0       Cardiovascular               0  
1          Respiratory               0  
2            Metabolic               0  
3       Cardiovascular               0  
4               Trauma               0  

[5 rows x 80 columns]

In [5]:
# Checking for duplicates
survival = survival.drop_duplicates()
survival.shape

(91713, 80)

In [6]:
# Column information
print(survival.columns)

Index(['age', 'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height',
       'icu_admit_source', 'icu_stay_type', 'icu_type', 'pre_icu_los_days',
       'weight', 'apache_2_diagnosis', 'apache_3j_diagnosis',
       'apache_post_operative', 'arf_apache', 'gcs_eyes_apache',
       'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache',
       'heart_rate_apache', 'intubated_apache', 'map_apache',
       'resprate_apache', 'temp_apache', 'ventilated_apache', 'd1_diasbp_max',
       'd1_diasbp_min', 'd1_diasbp_noninvasive_max',
       'd1_diasbp_noninvasive_min', 'd1_heartrate_max', 'd1_heartrate_min',
       'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max',
       'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resprate_min',
       'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min',
       'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'd1_temp_max',
       'd1_temp_min', 'h1_diasbp_max', 'h1_diasbp_min',
       'h1_diasbp_noninvasive_max', 'h1_diasbp_no

## Data Preprocessing

In [8]:
# Missing values analysis
missing_data = survival.isnull().sum()
missing_percentage = (missing_data / len(survival)) * 100

# Display columns with missing values and their percentage
missing_info = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_percentage}).round(2)
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)
print(missing_info)

                          Missing Values  Percentage
d1_potassium_min                    9585       10.45
d1_potassium_max                    9585       10.45
h1_mbp_noninvasive_min              9084        9.90
h1_mbp_noninvasive_max              9084        9.90
apache_4a_icu_death_prob            7947        8.67
...                                  ...         ...
d1_sysbp_min                         159        0.17
d1_heartrate_max                     145        0.16
d1_heartrate_min                     145        0.16
icu_admit_source                     112        0.12
gender                                25        0.03

[74 rows x 2 columns]


In [9]:
# Separate numeric and non-numeric columns
numeric_data = survival.select_dtypes(include=['number'])
non_numeric_data = survival.select_dtypes(exclude=['number'])
numeric_data.head()

age    bmi  elective_surgery  height  pre_icu_los_days  weight  \
0  68.0  22.73                 0   180.3          0.541667    73.9   
1  77.0  27.42                 0   160.0          0.927778    70.2   
2  25.0  31.95                 0   172.7          0.000694    95.3   
3  81.0  22.64                 1   165.1          0.000694    61.7   
4  19.0    NaN                 0   188.0          0.073611     NaN   

   apache_2_diagnosis  apache_3j_diagnosis  apache_post_operative  arf_apache  \
0               113.0               502.01                      0         0.0   
1               108.0               203.01                      0         0.0   
2               122.0               703.03                      0         0.0   
3               203.0              1206.03                      1         0.0   
4               119.0               601.01                      0         0.0   

   ...  apache_4a_icu_death_prob  aids  cirrhosis  diabetes_mellitus  \
0  ...                      0.05   0.0        0.0                1.0   
1  ...                      0.29   0.0        0.0                1.0   
2  ...                      0.00   0.0        0.0                0.0   
3  ...                      0.03   0.0        0.0                0.0   
4  ...                       NaN   0.0        0.0                0.0   

   hepatic_failure  immunosuppression  leukemia  lymphoma  \
0              0.0                0.0       0.0       0.0   
1              0.0                0.0       0.0       0.0   
2              0.0                0.0       0.0       0.0   
3              0.0                0.0       0.0       0.0   
4              0.0                0.0       0.0       0.0   

   solid_tumor_with_metastasis  hospital_death  
0                          0.0               0  
1                          0.0               0  
2                          0.0               0  
3                          0.0               0  
4                          0.0               0  

[5 rows x 73 columns]

In [10]:
# Non-numeric missing data
missing_data_non = non_numeric_data.isnull().sum()
missing_percentages_non = (missing_data_non / len(non_numeric_data)) * 100

# Display columns with missing values and their percentage
missing_info_non = pd.DataFrame({'Missing Values': missing_data_non, 'Percentage': missing_percentages_non}).round(2)
missing_info_non = missing_info_non[missing_info_non['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)
print(missing_info_non)


                      Missing Values  Percentage
apache_3j_bodysystem            1662        1.81
apache_2_bodysystem             1662        1.81
ethnicity                       1395        1.52
icu_admit_source                 112        0.12
gender                            25        0.03


## Imputation

In [12]:
# Loop through categorical columns to fill missing values with the mode
categorical_columns = ['apache_3j_bodysystem', 'apache_2_bodysystem', 'ethnicity', 'icu_admit_source', 'gender', 'icu_stay_type', 'icu_type']
for col in categorical_columns:
    mode_value = survival[col].mode()[0]
    print(mode_value,col)
    survival[col] = survival[col].fillna(mode_value)

Cardiovascular apache_3j_bodysystem
Cardiovascular apache_2_bodysystem
Caucasian ethnicity
Accident & Emergency icu_admit_source
M gender
admit icu_stay_type
Med-Surg ICU icu_type


In [13]:
# Loop through binary columns to fill missing values with the mode
binary_columns = ['arf_apache', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache', 'aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis'] 
for col in binary_columns:
    mode_value = survival[col].mode()[0]
    print(mode_value,col)
    survival[col] = survival[col].fillna(mode_value)

0.0 arf_apache
0.0 gcs_unable_apache
0.0 intubated_apache
0.0 ventilated_apache
0.0 aids
0.0 cirrhosis
0.0 diabetes_mellitus
0.0 hepatic_failure
0.0 immunosuppression
0.0 leukemia
0.0 lymphoma
0.0 solid_tumor_with_metastasis


In [14]:
# Loop through numeric-categorical columns to fill missing values with the mode
multicat_columns = ['gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache'] 
for col in multicat_columns:
    mode_value = survival[col].mode()[0]
    print(mode_value,col)
    survival[col] = survival[col].fillna(mode_value)

4.0 gcs_eyes_apache
6.0 gcs_motor_apache
5.0 gcs_verbal_apache


In [15]:
# List of numerical columns to fill missing values with the median
numerical_columns = ['age', 'bmi', 'height', 'weight']  
for col in numerical_columns:
    median_value = survival[col].median()
    print(median_value, col)
    survival[col] = survival[col].fillna(median_value)

65.0 age
27.65465458 bmi
170.1 height
80.3 weight


In [16]:
# apache columns
numerical_columns = ['apache_2_diagnosis', 'apache_3j_diagnosis', 'heart_rate_apache', 'map_apache','resprate_apache','temp_apache']  
for col in numerical_columns:
    median_value = survival[col].median()
    print(median_value, col)
    survival[col] = survival[col].fillna(median_value)

122.0 apache_2_diagnosis
409.02 apache_3j_diagnosis
104.0 heart_rate_apache
67.0 map_apache
28.0 resprate_apache
36.5 temp_apache


In [17]:
# other reading  columns
numerical_columns = ['d1_potassium_min', 'd1_potassium_max', 'heart_rate_apache', 'h1_mbp_max','h1_mbp_min','h1_diasbp_max','h1_diasbp_min','h1_sysbp_max','h1_sysbp_min','d1_glucose_min','d1_glucose_max','h1_resprate_max','h1_resprate_min','h1_heartrate_max','h1_heartrate_min','d1_temp_max','d1_temp_min','d1_diasbp_max','d1_diasbp_min','d1_resprate_min','d1_resprate_max','d1_spo2_min','d1_spo2_max','h1_spo2_max','h1_spo2_min','d1_mbp_min','d1_mbp_max','d1_sysbp_min','d1_sysbp_max','d1_heartrate_max','d1_heartrate_min']  
for col in numerical_columns:
    median_value = survival[col].median()
    print(median_value, col)
    survival[col] = survival[col].fillna(median_value)

3.9 d1_potassium_min
4.2 d1_potassium_max
104.0 heart_rate_apache
90.0 h1_mbp_max
78.0 h1_mbp_min
74.0 h1_diasbp_max
62.0 h1_diasbp_min
131.0 h1_sysbp_max
115.0 h1_sysbp_min
107.0 d1_glucose_min
150.0 d1_glucose_max
21.0 h1_resprate_max
16.0 h1_resprate_min
90.0 h1_heartrate_max
82.0 h1_heartrate_min
37.11 d1_temp_max
36.4 d1_temp_min
86.0 d1_diasbp_max
50.0 d1_diasbp_min
13.0 d1_resprate_min
26.0 d1_resprate_max
92.0 d1_spo2_min
100.0 d1_spo2_max
99.0 h1_spo2_max
96.0 h1_spo2_min
64.0 d1_mbp_min
102.0 d1_mbp_max
96.0 d1_sysbp_min
146.0 d1_sysbp_max
101.0 d1_heartrate_max
69.0 d1_heartrate_min


In [18]:
# List of columns to drop
columns_to_drop = ['apache_2_diagnosis','apache_3j_diagnosis','h1_mbp_noninvasive_min', 'h1_mbp_noninvasive_max', 'h1_diasbp_noninvasive_min','h1_diasbp_noninvasive_max','h1_sysbp_noninvasive_min','h1_sysbp_noninvasive_max','d1_mbp_noninvasive_max','d1_mbp_noninvasive_min','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min','d1_sysbp_noninvasive_min','d1_sysbp_noninvasive_max'] 

# Drop the columns and modify the DataFrame in place
survival.drop(columns=columns_to_drop, inplace=True)

In [19]:
survival['pre_icu_los_days'] = survival['pre_icu_los_days'].round(2)

In [20]:
negative_los = survival[survival['pre_icu_los_days'] < 0]
len(negative_los)

730

In [21]:
pos = survival[survival['pre_icu_los_days'] >= 0]
len(pos)

90983

In [22]:
df_full = survival.copy()

In [23]:
df_full.head()

age        bmi  elective_surgery  ethnicity gender  height  \
0  68.0  22.730000                 0  Caucasian      M   180.3   
1  77.0  27.420000                 0  Caucasian      F   160.0   
2  25.0  31.950000                 0  Caucasian      F   172.7   
3  81.0  22.640000                 1  Caucasian      F   165.1   
4  19.0  27.654655                 0  Caucasian      M   188.0   

            icu_admit_source icu_stay_type      icu_type  pre_icu_los_days  \
0                      Floor         admit         CTICU              0.54   
1                      Floor         admit  Med-Surg ICU              0.93   
2       Accident & Emergency         admit  Med-Surg ICU              0.00   
3  Operating Room / Recovery         admit         CTICU              0.00   
4       Accident & Emergency         admit  Med-Surg ICU              0.07   

   ...  cirrhosis  diabetes_mellitus  hepatic_failure  immunosuppression  \
0  ...        0.0                1.0              0.0                0.0   
1  ...        0.0                1.0              0.0                0.0   
2  ...        0.0                0.0              0.0                0.0   
3  ...        0.0                0.0              0.0                0.0   
4  ...        0.0                0.0              0.0                0.0   

   leukemia  lymphoma  solid_tumor_with_metastasis  apache_3j_bodysystem  \
0       0.0       0.0                          0.0                Sepsis   
1       0.0       0.0                          0.0           Respiratory   
2       0.0       0.0                          0.0             Metabolic   
3       0.0       0.0                          0.0        Cardiovascular   
4       0.0       0.0                          0.0                Trauma   

   apache_2_bodysystem  hospital_death  
0       Cardiovascular               0  
1          Respiratory               0  
2            Metabolic               0  
3       Cardiovascular               0  
4               Trauma               0  

[5 rows x 66 columns]

In [24]:
# Missing values analysis
missing_data = survival.isnull().sum()
missing_percentage = (missing_data / len(survival)) * 100

# Display columns with missing values and their percentage
missing_info = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_percentage}).round(2)
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)
print(missing_info)

                               Missing Values  Percentage
apache_4a_hospital_death_prob            7947        8.67
apache_4a_icu_death_prob                 7947        8.67


### Just for reference leaving the below code.

In [26]:
negative_los = numeric_data[numeric_data['pre_icu_los_days'] < 0]
len(negative_los)

778

In [27]:
pos = numeric_data[numeric_data['pre_icu_los_days'] >= 0]
len(pos)

90935

In [28]:
## Clean dataset

## SMOTE: Data Imbalance / Upsampling / Downsampling

### Outliers, Collinearity, Skewness (continuous, float), Summary statistics (continuous)

## Models

In [29]:
### Survial models: If time permits.

In [30]:
for column in survival.columns:
    print(f"Occurrences in '{column}':")
    print(survival[column].value_counts())
    print()  # Blank line for better readability

Occurrences in 'age':
age
65.0    6279
67.0    2271
68.0    2173
71.0    2141
72.0    2110
        ... 
19.0     343
20.0     338
18.0     259
17.0     126
16.0      44
Name: count, Length: 74, dtype: int64

Occurrences in 'bmi':
bmi
27.654655    3433
14.844926     443
67.814990     422
24.017768      87
24.208110      86
             ... 
23.241435       1
26.130528       1
24.463976       1
20.782525       1
47.179671       1
Name: count, Length: 34888, dtype: int64

Occurrences in 'elective_surgery':
elective_surgery
0    74862
1    16851
Name: count, dtype: int64

Occurrences in 'ethnicity':
ethnicity
Caucasian           72079
African American     9547
Other/Unknown        4374
Hispanic             3796
Asian                1129
Native American       788
Name: count, dtype: int64

Occurrences in 'gender':
gender
M    49494
F    42219
Name: count, dtype: int64

Occurrences in 'height':
height
167.6    5362
177.8    5289
160.0    5238
172.7    4834
165.1    4772
         ... 
173.6  

In [31]:
# Probability values: apache_4a_hospital_death_prob, apache_4a_icu_death_prob. Median!!

In [32]:
# Problematic columns: apache_2_diagnosis, apache_3j_diagnosis, Probability columns.

In [33]:
# Completed Mode imputation: ethnicity, gender, apache_3j_bodysystem, apache_2_bodysystem, icu_admit_source, 
# icu_stay_type, icu_type, aids, cirrhosis, diabetes_mellitus, hepatic_failure, gcs_unable_apache
# immunosuppression, leukemia, lymphoma, solid_tumor_with_metastasis, arf_apache, intubated_apache, 
# ventilated_apache, gcs_eyes_apache, gcs_motor_apache, , gcs_verbal_apache'

# Completed median imputation for all except probability columns.

In [34]:
# Things to keep in mind: apache_2_diagnosis, apache_3j_diagnosis are handled with median.
# Still need to take care of the 2 probability columns.

# Need clarity: pre_icu_los_days

In [35]:
# drop 4 (2 diagnosis + 2 bodysystem)
# drop 2 (just diagnosis numeric) ----------- apache_post_operative??
# Handle -1s for 2 probability columns. -1 -> NaNs -> mean/median imputation

In [36]:
# Filter the DataFrame for rows where hospital_death is 1
hospital_death_df = df_full[df_full['hospital_death'] == 1]
# Get the descriptive statistics for apache_4a_hospital_death_prob
print(hospital_death_df['apache_4a_hospital_death_prob'].describe())

count    7363.000000
mean        0.334838
std         0.354252
min        -1.000000
25%         0.130000
50%         0.300000
75%         0.580000
max         0.990000
Name: apache_4a_hospital_death_prob, dtype: float64


In [37]:
# Filter the DataFrame for rows where hospital_death is 1
hospital_death_df_0 = df_full[df_full['hospital_death'] == 0]
# Get the descriptive statistics for apache_4a_hospital_death_prob
print(hospital_death_df_0['apache_4a_hospital_death_prob'].describe())

count    76403.000000
mean         0.062882
std          0.220461
min         -1.000000
25%          0.020000
50%          0.040000
75%          0.110000
max          0.970000
Name: apache_4a_hospital_death_prob, dtype: float64


In [38]:
# Median values for each group
median_death_prob_1 = hospital_death_df['apache_4a_hospital_death_prob'].median()
median_death_prob_0 = hospital_death_df_0['apache_4a_hospital_death_prob'].median()

# Impute missing values based on hospital_death groups
df_full.loc[(df_full['hospital_death'] == 1) & (df_full['apache_4a_hospital_death_prob'].isna()), 'apache_4a_hospital_death_prob'] = median_death_prob_1
df_full.loc[(df_full['hospital_death'] == 0) & (df_full['apache_4a_hospital_death_prob'].isna()), 'apache_4a_hospital_death_prob'] = median_death_prob_0

In [39]:
# Missing values analysis
missing_data = df_full.isnull().sum()
missing_percentage = (missing_data / len(df_full)) * 100

# Display columns with missing values and their percentage
missing_info = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_percentage}).round(2)
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)
print(missing_info)

                          Missing Values  Percentage
apache_4a_icu_death_prob            7947        8.67


In [40]:
# Filter the DataFrame for rows where hospital_death is 1
hospital_death_4a_icu_df = df_full[df_full['hospital_death'] == 1]
# Get the descriptive statistics for apache_4a_hospital_death_prob
print(hospital_death_4a_icu_df['apache_4a_icu_death_prob'].describe())

count    7363.000000
mean        0.242726
std         0.325432
min        -1.000000
25%         0.060000
50%         0.180000
75%         0.435000
max         0.970000
Name: apache_4a_icu_death_prob, dtype: float64


In [41]:
# Filter the DataFrame for rows where hospital_death is 1
hospital_death_4a_icu_df_0 = df_full[df_full['hospital_death'] == 0]
# Get the descriptive statistics for apache_4a_hospital_death_prob
print(hospital_death_4a_icu_df_0['apache_4a_hospital_death_prob'].describe())

count    83798.000000
mean         0.060863
std          0.210609
min         -1.000000
25%          0.020000
50%          0.040000
75%          0.100000
max          0.970000
Name: apache_4a_hospital_death_prob, dtype: float64


In [42]:
# Median values for each group
median_icu_death_prob_1 = hospital_death_4a_icu_df['apache_4a_icu_death_prob'].median()
median_icu_death_prob_0 = hospital_death_4a_icu_df_0['apache_4a_icu_death_prob'].median()

# Impute missing values based on hospital_death groups
df_full.loc[(df_full['hospital_death'] == 1) & (df_full['apache_4a_icu_death_prob'].isna()), 'apache_4a_icu_death_prob'] = median_icu_death_prob_1
df_full.loc[(df_full['hospital_death'] == 0) & (df_full['apache_4a_icu_death_prob'].isna()), 'apache_4a_icu_death_prob'] = median_icu_death_prob_0


In [43]:
# Missing values analysis
missing_data = df_full.isnull().sum()
missing_percentage = (missing_data / len(df_full)) * 100

# Display columns with missing values and their percentage
missing_info = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_percentage}).round(2)
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)
print(missing_info)

Empty DataFrame
Columns: [Missing Values, Percentage]
Index: []


## Final columns check

In [45]:
df_full.shape

(91713, 66)

In [46]:
value_counts = df_full['hospital_death'].value_counts()
print(value_counts)

hospital_death
0    83798
1     7915
Name: count, dtype: int64


In [47]:
df_full.columns

Index(['age', 'bmi', 'elective_surgery', 'ethnicity', 'gender', 'height',
       'icu_admit_source', 'icu_stay_type', 'icu_type', 'pre_icu_los_days',
       'weight', 'apache_post_operative', 'arf_apache', 'gcs_eyes_apache',
       'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache',
       'heart_rate_apache', 'intubated_apache', 'map_apache',
       'resprate_apache', 'temp_apache', 'ventilated_apache', 'd1_diasbp_max',
       'd1_diasbp_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max',
       'd1_mbp_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max',
       'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_temp_max',
       'd1_temp_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_heartrate_max',
       'h1_heartrate_min', 'h1_mbp_max', 'h1_mbp_min', 'h1_resprate_max',
       'h1_resprate_min', 'h1_spo2_max', 'h1_spo2_min', 'h1_sysbp_max',
       'h1_sysbp_min', 'd1_glucose_max', 'd1_glucose_min', 'd1_potassium_max',
       'd1_potassium_min', 'apache_4a_hospi

In [48]:
df_full_categorical = ['apache_3j_bodysystem', 'apache_2_bodysystem', 'ethnicity', 
                       'icu_admit_source', 'gender', 'icu_stay_type', 'icu_type' ] 

df_full_binary = ['elective_surgery', 'arf_apache', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache', 
                  'aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure', 'immunosuppression', 'leukemia', 
                  'lymphoma', 'solid_tumor_with_metastasis', 'apache_post_operative'] 

df_full_multicat = ['gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache'] 

df_full_numeric = ['age', 'bmi', 'height', 'weight', 'pre_icu_los_days','heart_rate_apache', 'map_apache','resprate_apache',
                   'temp_apache', 'd1_potassium_min', 'd1_potassium_max', 'h1_mbp_max',
                   'h1_mbp_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_sysbp_max','h1_sysbp_min','d1_glucose_min',
                   'd1_glucose_max','h1_resprate_max','h1_resprate_min','h1_heartrate_max','h1_heartrate_min',
                   'd1_temp_max','d1_temp_min','d1_diasbp_max','d1_diasbp_min','d1_resprate_min','d1_resprate_max',
                   'd1_spo2_min','d1_spo2_max','h1_spo2_max','h1_spo2_min','d1_mbp_min','d1_mbp_max','d1_sysbp_min',
                   'd1_sysbp_max','d1_heartrate_max','d1_heartrate_min', 'apache_4a_hospital_death_prob',
                   'apache_4a_icu_death_prob']


In [49]:
# Assuming df is your DataFrame and 'target' is your target variable
X = df_full.drop('hospital_death', axis=1)  # Drop the target column
y = df_full['hospital_death']

In [51]:
df_full.to_csv("C:/Users/aksha/OneDrive/Desktop/MSBA_Semester_4/BUAN6341_AML/Project/Survival_Prediction_clean.csv", index=False)